In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'..') # https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder 
from sentinelhub import SHConfig, MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest
%matplotlib inline

def plot_image(image, factor=1.0, clip_range = None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])
    
# bboxfinder.com/#6.192609,-75.607738,6.301649,-75.538731 

betsiboka_coords_wgs84 =  [-75.607738,6.192609, -75.538731,6.301649]

CLIENT_ID = "5a0eee73-51af-4cee-b314-7169ce5798e5"
CLIENT_SECRET = "h@{#0;:P^Ol(xk04G#@Q%29r>?k2rrZ>IKg}4h:?"

config = SHConfig()

if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret=CLIENT_SECRET

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")
    
resolution = 10
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

In [2]:

evalscript = """
  
//VERSION=3
//Author: Maxim Lamare

function setup() {
  return {
    input: [
      {datasource: "S1GRD", bands:["VV", "VH"]},
      {datasource: "S2L2A", bands:["B02", "B03", "B08", "B04", "SCL"], units:"REFLECTANCE"}],
    output: [
      {id: "default", bands: 3, sampleType: SampleType.AUTO}
    ]
  }
}

function toDb(linear) {
  // Convert the linear backscatter to DB (Filgueiras et al. (2019), eq. 3)
  return 10 * Math.LN10 * linear
}

function calc_s1_ndvi(sigmaVV, sigmaVH){
    // Convert sigma0 to Decibels
    let vh_Db = toDb(sigmaVH)
    let vv_Db = toDb(sigmaVV)

    // Calculate NRPB (Filgueiras et al. (2019), eq. 4)
    let NRPB = (vh_Db - vv_Db) / (vh_Db + vv_Db)

    // Calculate NDVI_nc with approach A3 (Filgueiras et al. (2019), eq. 14)
    let NDVInc = 2.572 - 0.05047 * vh_Db + 0.176 * vv_Db + 3.422 * NRPB

    return NDVInc
}

function evaluatePixel(samples) {
  var s1 = samples.S1GRD[0]
  var s2 = samples.S2L2A[0]

  // Create an NDVI visualiser
  var viz=new ColorMapVisualizer([[0.0,0xa50026],
                                  [0.0,0xd73027], [0.2,0xf46d43],
                                  [0.3,0xfdae61], [0.4,0xfee08b],
                                  [0.5,0xffffbf], [0.6,0xd9ef8b],
                                  [0.7,0xa6d96a], [0.8,0x66bd63],
                                  [0.9,0x1a9850], [1.0,0x006837]]);
  // Calculate S2 NDVI
  let ndvi = index(s2.B08, s2.B04)
  // Calculate S1 NDVI
  let s1_ndvi = calc_s1_ndvi(s1.VV, s1.VH)

  // Use the S2-L2A classification to identify clouds
  if ([7, 8, 9, 10].includes(s2.SCL)) {
    // If clouds are present use S1 NDVI
    return {
      default: viz.process(s1_ndvi)
    }
  } else {
    // Otherwise use s2 NDVI
    return {
      default: viz.process(ndvi)
    }
  }
}
  
"""
bbox = BBox(bbox=[-75.606153, 6.176965, -75.536774, 6.313479], crs=CRS.WGS84)

request = SentinelHubRequest(
  evalscript=evalscript,
  input_data=[
    SentinelHubRequest.input_data(
      data_collection=DataCollection.SENTINEL1_IW,
      time_interval=('2021-07-07', '2021-07-15'),        
    ),
    SentinelHubRequest.input_data(
      data_collection=DataCollection.SENTINEL2_L2A,
      time_interval=('2021-07-07', '2021-07-15'),        
    ),
    
  ],
  responses=[
    SentinelHubRequest.output_response('default', MimeType.PNG),
    
  ],
  bbox=bbox,  
  size=[512, 1013.323],
  config=config
)
response = request.get_data() 

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"error":{"status":400,"reason":"Bad Request","message":"Dataset with id: S1GRD not found.","code":"RENDERER_EXCEPTION"}}"

In [ ]:
image = response[0]
plot_image(image, factor = 1/255, clip_range=(0,1))